Initiate the library


In [9]:
!pip install langchain openai chromadb unstructured pdfplumber google-search-results
#!pip install --upgrade langchain langchain-community openai
!pip install --upgrade langchain langchain-community langchain-openai openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00


Set Up API Keys in Colab



In [ ]:
import os

# Set API keys securely
os.environ["OPENAI_API_KEY"] = "" # OpenAI API
os.environ["SERPAPI_API_KEY"] = "" # Google Search API

Upload the Resume File in Google Colab


In [2]:
from google.colab import files

uploaded = files.upload()  # Uploads the file
resume_path = list(uploaded.keys())[0]  # Gets the file name
print(f"Uploaded file: {resume_path}")

Saving PM 2025 - Abnel Lluberes.pdf to PM 2025 - Abnel Lluberes.pdf
Uploaded file: PM 2025 - Abnel Lluberes.pdf


Extract Text from the Resume


In [3]:
# Install python-docx (if not already installed):

!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.7 MB/s eta 0:00:00


In [4]:
# Extract the text

import pdfplumber
import docx

def extract_text_from_resume(file_path):
    if file_path.lower().endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    elif file_path.lower().endswith(".docx"):
        doc = docx.Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text

resume_text = extract_text_from_resume(resume_path)
print("Extracted Resume Text (First 500 chars):\n", resume_text[:500])  # Preview

Extracted Resume Text (First 500 chars):
 Abnel Lluberes
Elmont, New York | (516) 552-2273| alluberes@gmail.com | alluberes.com
Profile
Results-driven Project Manager with 20+ years of experience in the software development and information
technology industry, leading innovative product strategies and complex, high-value projects. Demonstrated
expertise in project management, software engineering, and digital transformation. Holds an MBA and PMP
certified. Formation in risk management, quality management, and auditing, and a proven trac


Step 4: Analyze Skills and Competencies from the Resume

In [10]:
#from langchain.chat_models import ChatOpenAI
#from langchain.prompts import PromptTemplate
#from langchain.chains import LLMChain

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Initialize OpenAI model
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["resume_text"],
    template="""Extract the following details from the given resume text:
    - List of key skills (technical & soft skills)
    - Work experiences (company, position, duration)
    - Education details (degree, institution, year)
    - Certifications (if any)
    - Summary of expertise

    Resume Text:
    {resume_text}

    Provide the extracted details in a structured format.
    """
)

# Create the chain
resume_analysis_chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain on the extracted resume text
analysis_result = resume_analysis_chain.run(resume_text)

print("Extracted Resume Details:\n", analysis_result)

Extracted Resume Details:
 - Key Skills:
    - Technical Skills: Project Management, Software Engineering, Digital Transformation, C# Programming, Azure resource administration, Agile Methodologies, Quality Management, Risk Management, Auditing
    - Soft Skills: Cross-location collaboration

- Work Experiences:
    - Senior Product Manager at TripleCyber, from Jan 2022 to Jan 2025
    - Project Manager at Agilisa Technologies, from Mar 2020 to Dec 2021
    - Business Advisor at CMTD-ITLA, from Mar 2019 to Feb 2020
    - Project Manager at SENASA, from May 2012 to Jul 2018

- Education Details:
    - Master in Business Administration from UASD, Dominican Republic in Nov 2019
    - System Engineer from O&M University, Dominican Republic in Jan 2008
    - PMP (2236059) from Project Management Institute in Jul 2018

- Certifications: PMP (2236059) from Project Management Institute in Jul 2018

- Summary of Expertise: Results-driven Project Manager with 20+ years of experience in the softw

In [11]:
# Make sure SerpAPI is installed:

!pip install google-search-results

Then, import the necessary libraries:

In [14]:
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper

# Initialize SerpAPI for job search
search = SerpAPIWrapper()

Create a Function to Search for Jobs


In [16]:
def search_jobs(query, location="remote", num_results=5):
    search_query = f"{query} jobs in {location}"

    # results = search.results(search_query, num_results)
    results = search.results(query=search_query)

    job_listings = []
    for result in results["organic_results"]:
        job_listings.append({
            "title": result["title"],
            "company": result["source"],
            "link": result["link"]
        })

    return job_listings

# Example: Search for jobs using extracted skills
skills_query = analysis_result  # Using the extracted skills from the resume
jobs = search_jobs(skills_query)

# Display job results
for idx, job in enumerate(jobs, start=1):
    print(f"{idx}. {job['title']} at {job['company']}")
    print(f"Link: {job['link']}\n")

1. Resume Skills for Project Manager (+ Templates) at Resume Worded
Link: https://resumeworded.com/skills-and-keywords/project-manager-skills

2. 11 Software Project Manager Resume Examples for 2025 at CV Compiler
Link: https://cvcompiler.com/software-project-manager-resume-examples

3. Resume Skills for Information Technology Manager ... at Resume Worded
Link: https://resumeworded.com/skills-and-keywords/information-technology-manager-skills

4. What core technical skills are needed by an Agile Project ... at Quora
Link: https://www.quora.com/What-core-technical-skills-are-needed-by-an-Agile-Project-Manager

5. Cloud Project Manager Resume Washington, DC at Hire IT People
Link: https://www.hireitpeople.com/resume-database/81-project-manager-resumes/250787-cloud-project-manager-resume-washington-dc

6. Software/IT Project Managers - key skills required? at Reddit · r/projectmanagement
Link: https://www.reddit.com/r/projectmanagement/comments/jkauab/softwareit_project_managers_key_skill

Step 6: Customize the Resume for a Job Listing


In [17]:
# 1. Select a Job Listing
# Manually select a job listing (e.g., the first one)
selected_job = jobs[0]  # Change index if needed

print(f"Selected Job: {selected_job['title']} at {selected_job['company']}")
print(f"Job Link: {selected_job['link']}\n")

Selected Job: Resume Skills for Project Manager (+ Templates) at Resume Worded
Job Link: https://resumeworded.com/skills-and-keywords/project-manager-skills



In [19]:
# 2. Generate a Tailored Resume

# Define a prompt template for resume optimization
resume_prompt = PromptTemplate(
    input_variables=["resume_text", "job_title", "company", "job_description"],
    template="""Based on the given resume, tailor a new version that highlights
    relevant skills and experience for the following job:

    Job Title: {job_title}
    Company: {company}
    Job Description: {job_description}

    Original Resume:
    {resume_text}

    Generate a new, optimized resume for submission.
    """
)

# Create a LangChain chain
resume_chain = LLMChain(llm=llm, prompt=resume_prompt)

# Simulated job description (if not available in search results)
job_description = "We are looking for a skilled software developer with experience in Python, AI, and cloud computing."

# Generate the tailored resume
tailored_resume = resume_chain.run({
    "resume_text": resume_text,
    "job_title": selected_job["title"],
    "company": selected_job["company"],
    "job_description": job_description
})

print("Tailored Resume:\n", tailored_resume)

Tailored Resume:
 Abnel Lluberes
Elmont, New York | (516) 552-2273| alluberes@gmail.com | alluberes.com

Profile
Experienced Software Developer and Project Manager with over 20 years of experience in the software development and information technology industry. Proven expertise in Python, AI, and cloud computing, with a strong background in project management, software engineering, and digital transformation. Holds an MBA and PMP certification. Skilled in risk management, quality management, and auditing, with a track record of driving business effectiveness.

Experience
SENIOR PRODUCT MANAGER | TRIPLECYBER | JAN 2022 – JAN 2025
• Managed teams and refined procedures for various departments, ensuring smooth administrative operations.
• Oversaw the product and project roadmap, reporting to the CEO and delivering to the development team.
• Fostered a collaborative environment among multiple cultural teams by addressing language barriers and promoting open communication channels.
• Design

Step 7: Save the Tailored Resume as a PDF or DOCX


In [20]:
# Install Dependencies (if needed)

!pip install python-docx reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.5 MB/s eta 0:00:00


In [21]:
from docx import Document

def save_resume_as_docx(text, filename="tailored_resume.docx"):
    doc = Document()
    doc.add_paragraph(text)
    doc.save(filename)
    print(f"Resume saved as {filename}")

# Save the tailored resume as a Word document
save_resume_as_docx(tailored_resume)

Resume saved as tailored_resume.docx


In [22]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def save_resume_as_pdf(text, filename="tailored_resume.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    # Split text into lines and write to PDF
    lines = text.split("\n")
    y_position = height - 40  # Start position

    for line in lines:
        c.drawString(40, y_position, line)
        y_position -= 15  # Move down for next line

    c.save()
    print(f"Resume saved as {filename}")

# Save the tailored resume as a PDF
save_resume_as_pdf(tailored_resume)

Resume saved as tailored_resume.pdf
